In [ ]:
# default_exp core

# nbdev_try2.core

> Give it another try.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Sara")

'Hello Sara!'

## A Little Database

Let's try how to use a database in a way that it will not break CI tests on GitHub...

In [ ]:
import sqlite3
import os

In [ ]:
DBNAME = "example.sqlite"
if os.path.exists(DBNAME):
    print("removing", DBNAME)
    os.remove(DBNAME)
    
conn = sqlite3.connect(DBNAME)
cur = conn.cursor()
cur.execute("""
    create table if not exists numbers (
        number integer primary key,
        name
    )
""")
cur.execute("""
    insert into numbers (number, name) 
        values (1, "One"),
            (2, "Two"),
            (3, "Three"),
            (17, "Seventeen")
""")
resu = cur.execute("""
    select name from numbers where number = ?
""", (17, )).fetchall()

assert len(resu) == 1, len(resu)
assert resu[0][0] == "Seventeen"

print(resu)

removing example.sqlite
[('Seventeen',)]


This is working. Also repetitive, because it removes tha database before. So let's try to push it...

Tests succeeded in GitHub, but we have no clue whether it was executed :(

So let's try with something failing...

In [ ]:
# assert 1 == 0, "deliberately failing"

This actually passes `nbdev_test_nbs` successfully.

Docs say:

> Everything that is not an exported cell is considered a test

So, again, something plain wrong :(

Cool, [in GitHub it fails](https://github.com/flaschbier/nbdev_try2/runs/543541993) properly. But this is even worse, when "tests" run locally but fail in CI.

Now it also fails locally. What kind of ordering mess due to the context change Jupyter vs. shell is now going on? On the other hand we see that at least a small file is possible in GitHub CI.

Bah, it "sometimes" requires saving. But this will create a checkpoint also, which is not wanted. Here the design flaws fully strike back.

Findings:
- GitHub CI allows for (at least small) file system consumption
- all cells not markes are treated as tests
- saving required to make the shell tools aware of souce code changes

## Now For Requests

FTP will be added soon, no worries...

In [ ]:
import requests
r = requests.get("https://httpbin.org/get", headers={"accept": "application/json"})
assert r.status_code == 200, r.status_code

print(r.json())
assert r.json()["url"] == 'https://httpbin.org/get'

{'args': {}, 'headers': {'Accept': 'application/json', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.21.0', 'X-Amzn-Trace-Id': 'Root=1-5e80db62-683330dce17be93ca5dc2f70'}, 'origin': '46.5.69.8', 'url': 'https://httpbin.org/get'}
